In [1]:
%matplotlib inline 

In [2]:
import json
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

In [3]:
def CustomParser(data):
    return json.loads(data)

# Read csv file
filename = 'result.csv'
df = pd.read_csv(filename, sep=';', converters={'db_parameters':CustomParser, 'model_cfg': CustomParser})
df[sorted(df['db_parameters'][0].keys())] = df['db_parameters'].apply(pd.Series)
df[sorted(df['model_cfg'][0].keys())] = df['model_cfg'].apply(pd.Series)

In [10]:
import os
FOLDER='../results/'

folders = set(os.listdir(FOLDER))
folders.remove('_sources')

result = {
    'acc':[],
    'f1':[],
    'db':[],
    'task':[],
    'model':[]
}

for root_folder in folders:
    for run_id in os.listdir(FOLDER+root_folder):
        current_folder = FOLDER+root_folder+'/'+run_id

        metrics_file = current_folder+'/metrics-test.csv'
        config_file = current_folder+'/config.json'
        
        if not os.path.exists(metrics_file) or not os.path.exists(config_file):
            continue

        config_df = pd.read_json(config_file)
        if config_df['dataset_cfg']['reader'] != "pypagai.preprocessing.dataset_babi.BaBIDataset":
            continue
            
        db = 'babi' 

        task = config_df['dataset_cfg']['task']
        model = config_df['model_default_cfg']['model'].split('.')[-1]
        
        metrics_df = pd.read_csv(metrics_file)
        acc = metrics_df['accuracy'][0]
        f1 = metrics_df['f1_micro'][0]
        
        result['f1'].append(f1)
        result['db'].append(db)
        result['acc'].append(acc)
        result['task'].append(task)
        result['model'].append(model)
        result['db_parameters'] = config_file

        
new_results = pd.DataFrame(result)

In [11]:
new_results

,acc,db,db_parameters,f1,model,task
0,0.470,babi,../results/model_cfg.model=pypagai.models.mode...,0.470,SimpleLSTM,6
1,0.437,babi,../results/model_cfg.model=pypagai.models.mode...,0.437,SimpleLSTM,10
2,0.638,babi,../results/model_cfg.model=pypagai.models.mode...,0.638,SimpleLSTM,9
3,0.531,babi,../results/model_cfg.model=pypagai.models.mode...,0.531,SimpleLSTM,18
4,0.213,babi,../results/model_cfg.model=pypagai.models.mode...,0.213,SimpleLSTM,15
5,0.475,babi,../results/model_cfg.model=pypagai.models.mode...,0.475,SimpleLSTM,20
6,0.172,babi,../results/model_cfg.model=pypagai.models.mode...,0.172,SimpleLSTM,3
7,0.187,babi,../results/model_cfg.model=pypagai.models.mode...,0.187,SimpleLSTM,11
8,0.573,babi,../results/model_cfg.model=pypagai.models.mode...,0.573,SimpleLSTM,17
9,0.248,babi,../results/model_cfg.model=pypagai.models.mode...,0.248,SimpleLSTM,13


In [13]:
# Show best result of each model for each babi task
ddf = df[df['db'] == 'babi']
ddf = pd.concat([new_results, ddf], keys=['acc','f1','db','task','model'])

ddf = df[df['db'] == 'babi']
ddf = ddf.groupby(['db','model','task'])
ddf = ddf.max()
ddf = ddf.reset_index()
ddf[['model','acc','task']].pivot('task', 'model')

acc                                      
model DeepN2NMemory EmbedLSTM N2NMemory     RN SimpleLSTM
task                                                     
1             0.963     0.711     0.953    NaN      0.189
2             0.325       NaN     0.281    NaN      0.189
3             0.251       NaN     0.209    NaN      0.174
4             1.000       NaN     0.958    NaN      0.279
5             0.826       NaN     0.859    NaN      0.344
6             0.972       NaN     0.867    NaN      0.636
7             0.904       NaN     0.855    NaN      0.693
8             0.802       NaN     0.692    NaN      0.539
9             0.962       NaN     0.792    NaN      0.655
10            0.956       NaN     0.910    NaN      0.538
11            0.985       NaN     0.961    NaN      0.220
12            0.967       NaN     0.957    NaN      0.187
13            1.000       NaN     0.952  0.907      0.185
14            0.412       NaN     0.364  0.449      0.205
15            0.536       NaN     0.478  0.597      0.266
16            0.451       NaN     0.456  0.454      0.281
17            0.958       NaN     0.850  0.662      0.612
18            0.957       NaN     0.930  0.972      0.709
19            0.086       NaN     0.126  0.151      0.094
20            0.983       NaN     0.982  0.976      0.699